<a href="https://colab.research.google.com/github/BaronVonBussin/Stuff/blob/main/fisher_transform_20241220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class KalmanFilter:
    def __init__(self, R=1, Q=0.1):
        self.R = R  # measurement noise
        self.Q = Q  # process noise
        self.P = 1  # initial covariance
        self.x = None  # initial state

    def filter(self, measurement):
        if self.x is None:
            self.x = measurement
            return measurement

        # Predict
        x_pred = self.x
        P_pred = self.P + self.Q

        # Update
        K = P_pred / (P_pred + self.R)  # Kalman gain
        self.x = x_pred + K * (measurement - x_pred)
        self.P = (1 - K) * P_pred

        return self.x

def fisher_transform(series, period=10):
    """
    Compute Fisher Transform on a price series with better handling of edge cases
    """
    # Calculate price changes
    changes = series.diff(1).rolling(window=period).sum()

    # Calculate max and min with enough buffer to avoid division by zero
    max_change = changes.rolling(window=period).max()
    min_change = changes.rolling(window=period).min()

    # Add small epsilon to avoid division by zero
    epsilon = 1e-10
    denominator = (max_change - min_change) + epsilon

    # Normalize to -0.999 to +0.999 range (avoiding exact ±1)
    normalized = (changes - min_change) / denominator
    normalized = (normalized * 1.998) - 0.999  # Scale to -0.999 to 0.999

    # Apply Fisher Transform with safeguards
    normalized = normalized.clip(-0.999, 0.999)  # Ensure we stay within bounds
    fisher = 0.5 * np.log((1 + normalized) / (1 - normalized))

    return fisher

# Load and process data
try:
    df = pd.read_csv('/content/SPX_Data.csv')
    df['Date'] = pd.to_datetime(df['Date'])

    # Apply Kalman Filter
    kf = KalmanFilter(R=0.01, Q=1.0)  # Adjusted parameters for better trend following
    df['Kalman'] = df['Close'].apply(kf.filter)

    # Apply Fisher Transform
    df['Fisher'] = fisher_transform(df['Close'], period=10)

    # Create visualization
    fig = make_subplots(rows=3, cols=1,
                    row_heights=[0.5, 0.25, 0.25],
                    shared_xaxes=True,
                    vertical_spacing=0.05)

# Price and Kalman
    fig.add_trace(
        go.Candlestick(
            x=df['Date'],
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='OHLC',
            xperiod='D',  # Daily spacing
            xperiodalignment='start'
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=df['Date'],
               y=df['Kalman'],
               name='Kalman',
               line=dict(color='red')),
        row=1, col=1
    )


    # Fisher Transform
    fig.add_trace(
        go.Scatter(x=df['Date'], y=df['Fisher'], name='Fisher Transform',
                  line=dict(color='green')),
        row=2, col=1
    )
    # Add horizontal lines at +2 and -2 standard deviations
    fisher_std = df['Fisher'].std()
    fig.add_hline(y=2*fisher_std, line_dash="dash", line_color="gray", row=2, col=1)
    fig.add_hline(y=-2*fisher_std, line_dash="dash", line_color="gray", row=2, col=1)

    # Price Changes
    df['Returns'] = df['Close'].pct_change()
    fig.add_trace(
        go.Scatter(x=df['Date'], y=df['Returns'], name='Returns',
                  line=dict(color='purple')),
        row=3, col=1
    )

    # Update layout
    fig.update_layout(
        title='SPY: Kalman Filter vs Fisher Transform Comparison',
        height=1000,
        showlegend=True
    )

    # Add range selector
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        row=3, col=1
    )

    fig.show()

    # Print some analytical comparisons
    print("\nKey differences in signals:")

    # Find major turning points in each
    kalman_changes = df['Kalman'].diff().abs()
    fisher_changes = df['Fisher'].diff().abs()

    print("\nMajor Kalman Filter changes (top 5):")
    major_k = df[kalman_changes > kalman_changes.quantile(0.95)].sort_values('Date')
    for _, row in major_k.head().iterrows():
        print(f"Date: {row['Date'].strftime('%Y-%m-%d')}, Price: ${row['Close']:.2f}")

    print("\nMajor Fisher Transform signals (top 5):")
    major_f = df[abs(df['Fisher']) > 2*fisher_std].sort_values('Date')
    for _, row in major_f.head().iterrows():
        print(f"Date: {row['Date'].strftime('%Y-%m-%d')}, Price: ${row['Close']:.2f}")

except Exception as e:
    print(f"Error: {str(e)}")


Key differences in signals:

Major Kalman Filter changes (top 5):
Date: 2013-01-02, Price: $1462.42
Date: 2013-04-15, Price: $1552.36
Date: 2013-06-20, Price: $1588.19
Date: 2013-10-10, Price: $1692.56
Date: 2014-01-24, Price: $1790.29

Major Fisher Transform signals (top 5):
